In [139]:
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Flatten, Activation, Reshape, Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2
from keras.layers.merge import add, concatenate
from keras.engine.topology import Layer
from keras.engine import InputSpec
from keras.utils import conv_utils
from tensorflow.python.ops import array_ops
from utils.BilinearUpSampling import BilinearUpSampling2D

import numpy as np
import scipy.misc as misc
import os
import glob
from random import shuffle, randint

import matplotlib.pyplot as plt

In [118]:
def resize_images(x, size, method='bilinear'):
    new_size = tf.convert_to_tensor(size, dtype=tf.int32)
    resized = tf.image.resize_images(x, new_size)
    return resized

In [119]:
class BilinearUpSampling2D(Layer):
    """Upsampling2D with bilinear interpolation."""

    def __init__(self, target_shape=None, data_format=None, **kwargs):
        if data_format is None:
            data_format = K.image_data_format()
        assert data_format in {
            'channels_last', 'channels_first'}
        self.data_format = data_format
        self.input_spec = [InputSpec(ndim=4)]
        self.target_shape = target_shape
        if self.data_format == 'channels_first':
            self.target_size = (target_shape[2], target_shape[3])
        elif self.data_format == 'channels_last':
            self.target_size = (target_shape[1], target_shape[2])
        super(BilinearUpSampling2D, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], self.target_size[0],
                    self.target_size[1], input_shape[3])
        else:
            return (input_shape[0], input_shape[1],
                    self.target_size[0], self.target_size[1])

    def call(self, inputs):
        return resize_images(inputs, size=self.target_size,
                             method='bilinear')

    def get_config(self):
        config = {'target_shape': self.target_shape,
                'data_format': self.data_format}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class CroppingLike2D(Layer):
    def __init__(self, target_shape, offset=None, data_format=None,
                 **kwargs):
        """Crop to target.
        If only one `offset` is set, then all dimensions are offset by this amount.
        """
        super(CroppingLike2D, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.target_shape = target_shape
        if offset is None or offset == 'centered':
            self.offset = 'centered'
        elif isinstance(offset, int):
            self.offset = (offset, offset)
        elif hasattr(offset, '__len__'):
            if len(offset) != 2:
                raise ValueError('`offset` should have two elements. '
                                 'Found: ' + str(offset))
            self.offset = offset
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            return (input_shape[0],
                    input_shape[1],
                    self.target_shape[2],
                    self.target_shape[3])
        else:
            return (input_shape[0],
                    self.target_shape[1],
                    self.target_shape[2],
                    input_shape[3])

    def call(self, inputs):
        input_shape = K.int_shape(inputs)
        if self.data_format == 'channels_first':
            input_height = input_shape[2]
            input_width = input_shape[3]
            target_height = self.target_shape[2]
            target_width = self.target_shape[3]
            if target_height > input_height or target_width > input_width:
                raise ValueError('The Tensor to be cropped need to be smaller'
                                 'or equal to the target Tensor.')

            if self.offset == 'centered':
                self.offset = [int((input_height - target_height) / 2),
                               int((input_width - target_width) / 2)]

            if self.offset[0] + target_height > input_height:
                raise ValueError('Height index out of range: '
                                 + str(self.offset[0] + target_height))
            if self.offset[1] + target_width > input_width:
                raise ValueError('Width index out of range:'
                                 + str(self.offset[1] + target_width))

            return inputs[:,
                          :,
                          self.offset[0]:self.offset[0] + target_height,
                          self.offset[1]:self.offset[1] + target_width]
        elif self.data_format == 'channels_last':
            input_height = input_shape[1]
            input_width = input_shape[2]
            target_height = self.target_shape[1]
            target_width = self.target_shape[2]
            if target_height > input_height or target_width > input_width:
                raise ValueError('The Tensor to be cropped need to be smaller'
                                 'or equal to the target Tensor.')

            if self.offset == 'centered':
                self.offset = [int((input_height - target_height) / 2),
                               int((input_width - target_width) / 2)]

            if self.offset[0] + target_height > input_height:
                raise ValueError('Height index out of range: '
                                 + str(self.offset[0] + target_height))
            if self.offset[1] + target_width > input_width:
                raise ValueError('Width index out of range:'
                                 + str(self.offset[1] + target_width))
            output = inputs[:,
                            self.offset[0]:self.offset[0] + target_height,
                            self.offset[1]:self.offset[1] + target_width,
                            :]
            return output

    def get_config(self):
        config = {'target_shape': self.target_shape,
                  'offset': self.offset,
                  'data_format': self.data_format}
        base_config = super(CroppingLike2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [120]:
def vgg_conv(filters, convs, padding=False, weight_decay=0.,
             block_name='blockx'):
    def f(x):
        for i in range(convs):
            if block_name == 'block1' and i == 0:
                if padding is True:
                    x = ZeroPadding2D(padding=(100, 100))(x)
                x = Conv2D(filters, (3,3), activation='relu', padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(weight_decay),
                           name='{}_conv{}'.format(block_name, int(i+1)))(x)
            else:
                x = Conv2D(filters, (3,3), activation='relu', padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(weight_decay),
                           name='{}_conv{}'.format(block_name, int(i+1)))(x)
        pool = MaxPooling2D((2,2), strides=(2,2), padding='same',
                            name='{}_pool'.format(block_name))(x)
        dropout = Dropout(0.85)(pool)
        return dropout
    return f

def vgg_fc(filters, weight_decay=0., block_name='block5'):
    def f(x):
        fc6 = Conv2D(filters=4096, kernel_size=(3, 3),
                     activation='relu', padding='same',
                     dilation_rate=(2, 2),
                     kernel_initializer='he_normal',
                     kernel_regularizer=l2(weight_decay),
                     name='{}_fc6'.format(block_name))(x)
        drop6 = Dropout(0.85)(fc6)
        return drop6
    return f

def vgg_deconv(classes, scale=1, kernel_size=(4, 4), strides=(2, 2), target_shape=None,
               crop_offset='centered', weight_decay=0., block_name='featx'):
    def f(x, y):
        def scaling(xx, ss=1):
            return xx * ss
        scaled = Lambda(scaling, arguments={'ss': scale},
                        name='scale_{}'.format(block_name))(x)
        if y is None:
            upscore = Conv2DTranspose(filters=classes, kernel_size=kernel_size,
                                      strides=strides, padding='same',
                                      kernel_initializer='he_normal',
                                      kernel_regularizer=l2(weight_decay),
                                      use_bias=True,
                                      name='upscore_{}'.format(block_name))(scaled)
            upsample = BilinearUpSampling2D(target_shape=target_shape,
                                            name='upsample_{}'.format(block_name))(upscore)
        else:
            crop = CroppingLike2D(target_shape=K.int_shape(y),
                                 name='crop_{}'.format(block_name))(scaled)
            merge = concatenate([y, crop])
    
            fuse1 = Conv2D(filters=2*classes, kernel_size=(1,1), 
                           activation='relu', name='upscore_{}_fuse_1'.format(block_name))(merge)
            fuse2 = Conv2D(filters=classes, kernel_size=(1,1), 
                           activation='relu', name='upscore_{}_fuse_2'.format(block_name))(fuse1)
            upscore = Conv2DTranspose(filters=classes, kernel_size=kernel_size,
                                      strides=strides, padding='same',
                                      kernel_initializer='he_normal',
                                      kernel_regularizer=l2(weight_decay),
                                      use_bias=True,
                                      name='upscore_{}'.format(block_name))(fuse2)
            upsample = BilinearUpSampling2D(target_shape=target_shape,
                                            name='upsample_{}'.format(block_name))(upscore)
        return upsample
    return f

def vgg_upsampling(classes, target_shape=None, scale=1, weight_decay=0., block_name='featx'):
    def f(x, y):
        score = Conv2D(filters=classes, kernel_size=(1, 1),
                       activation='linear',
                       padding='valid',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(weight_decay),
                       name='score_{}'.format(block_name))(x)
        if y is not None:
            def scaling(xx, ss=1):
                return xx * ss
            scaled = Lambda(scaling, arguments={'ss': scale},
                            name='scale_{}'.format(block_name))(score)
            score = add([y, scaled])
        upscore = BilinearUpSampling2D(
            target_shape=target_shape,
            name='upscore_{}'.format(block_name))(score)
        return upscore
    return f

def vgg_score(crop_offset='centered'):
    def f(x, y):
        score = CroppingLike2D(target_shape=K.int_shape(
            x), offset=crop_offset, name='score')(y)
        return score
    return f

# Encoder

In [121]:
class Encoder(Model):
    def __init__(self, inputs, blocks, 
                 weights=None, trainable=True,
                 name='encoder'):
        inverse_pyramid = []
        
        conv_blocks = blocks[:-1]
        for i, block in enumerate(conv_blocks):
            if i == 0:
                x = block(inputs)
                inverse_pyramid.append(x)
            elif i < len(conv_blocks) - 1:
                x = block(x)
                inverse_pyramid.append(x)
            else:
                x = block(x)
        
        # fully convolutional block
        fc_block = blocks[-1]
        y = fc_block(x)
        inverse_pyramid.append(y)
        
        # Reverse the pyramid features
        outputs = list(reversed(inverse_pyramid))
        
        self.inputs = inputs
        self.outputs = outputs

# Decoder

In [122]:
def Decoder(pyramid, blocks):
    if len(blocks) != len(pyramid):
        raise ValueError('`blocks` needs to match the length of'
                         '`pyramid`.')
    
    decoded = None
    for feat, blk in zip(pyramid, blocks):
        print(feat, decoded, blk)
        decoded = blk(feat, decoded)
    return decoded

# FCN

In [123]:
inputs = Input(shape=(250, 250, 1))

In [124]:
blocks = [vgg_conv(64, 1, block_name='block1'),
          vgg_conv(128, 1, block_name='block2'),
          vgg_conv(256, 1, block_name='block3'),
          vgg_conv(512, 1, block_name='block4'),
          vgg_conv(512, 1, block_name='block5'),
          vgg_fc(4096)]

In [125]:
encoder = Encoder(inputs, blocks)

In [126]:
feat_pyramid = encoder.outputs   # A feature pyramid with 5 scales
feat_pyramid = feat_pyramid[:5]  # Select only the top three scale of the pyramid
feat_pyramid.append(inputs)      # Add image to the bottom of the pyramid

In [127]:
feat_pyramid

[<tf.Tensor 'dropout_6/cond/Merge:0' shape=(?, 8, 8, 4096) dtype=float32>,
 <tf.Tensor 'dropout_4/cond/Merge:0' shape=(?, 16, 16, 512) dtype=float32>,
 <tf.Tensor 'dropout_3/cond/Merge:0' shape=(?, 32, 32, 256) dtype=float32>,
 <tf.Tensor 'dropout_2/cond/Merge:0' shape=(?, 63, 63, 128) dtype=float32>,
 <tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 125, 125, 64) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 250, 250, 1) dtype=float32>]

In [128]:
decode_blocks = [
    vgg_deconv(classes=512, target_shape=(None, 16, 16, None), kernel_size=(4,4), strides=(2,2), block_name='feat1'),
    vgg_deconv(classes=256, target_shape=(None, 32, 32, None), kernel_size=(4,4), strides=(2,2), block_name='feat2'),
    vgg_deconv(classes=128, target_shape=(None, 63, 63, None), kernel_size=(4,4), strides=(2,2), block_name='feat3'),
    vgg_deconv(classes=124, target_shape=(None, 125, 125, None), kernel_size=(4,4), strides=(2,2), block_name='feat4'),
    vgg_deconv(classes=124, target_shape=(None, 250, 250, None), kernel_size=(16,16), strides=(2,2), block_name='feat5'),
]

outputs = Decoder(pyramid=feat_pyramid[:-1], blocks=decode_blocks)

Tensor("dropout_6/cond/Merge:0", shape=(?, 8, 8, 4096), dtype=float32) None <function vgg_deconv.<locals>.f at 0x1822d497b8>
Tensor("dropout_4/cond/Merge:0", shape=(?, 16, 16, 512), dtype=float32) Tensor("upsample_feat1/ResizeBilinear:0", shape=(?, 16, 16, 512), dtype=float32) <function vgg_deconv.<locals>.f at 0x1822d49ea0>
Tensor("dropout_3/cond/Merge:0", shape=(?, 32, 32, 256), dtype=float32) Tensor("upsample_feat2/ResizeBilinear:0", shape=(?, 32, 32, 256), dtype=float32) <function vgg_deconv.<locals>.f at 0x1823117b70>
Tensor("dropout_2/cond/Merge:0", shape=(?, 63, 63, 128), dtype=float32) Tensor("upsample_feat3/ResizeBilinear:0", shape=(?, 63, 63, 128), dtype=float32) <function vgg_deconv.<locals>.f at 0x18231179d8>
Tensor("dropout_1/cond/Merge:0", shape=(?, 125, 125, 64), dtype=float32) Tensor("upsample_feat4/ResizeBilinear:0", shape=(?, 125, 125, 124), dtype=float32) <function vgg_deconv.<locals>.f at 0x1823117ea0>


In [129]:
# outputs = Activation('softmax')(outputs)

In [131]:
model = Model(inputs=inputs, outputs=outputs)
plot_model(model, show_shapes=True, to_file='orig_model.png')

In [ ]:
adam = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)

def customLoss(target, output):
    targets = tf.squeeze(K.cast(target, 'int64'), squeeze_dims=[3])
    res = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets, logits=output)
    return tf.reduce_mean(res)


In [ ]:
model.compile(optimizer=adam,
              loss=customLoss,
              metrics=['accuracy'])

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [145]:
input = Input(shape=(250, 250, 1))
n_classes = 124

In [146]:
conv2 = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name='conv2')(input)
pool2 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool2')(conv2)
dropout2 = Dropout(rate=0.85, name='dropout2')(pool2)

conv3 = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name='conv3')(dropout2)
pool3 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool3')(conv3)
dropout3 = Dropout(rate=0.85, name='dropout3')(pool3)

conv4 = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', name='conv4')(dropout3)
pool4 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool4')(conv4)
dropout4 = Dropout(rate=0.85, name='dropout4')(pool4)

conv5 = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='conv5')(dropout4)
pool5 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool5')(conv5)
dropout5 = Dropout(rate=0.85, name='dropout5')(pool5)

conv6 = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='conv6')(dropout5)
pool6 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool6')(conv6)
dropout6 = Dropout(rate=0.85, name='dropout6')(pool6)

conv7 = Conv2D(filters=4096, kernel_size=(3,3), padding='same', name='conv7')(dropout6)

In [147]:
conv_t1 = Conv2DTranspose(filters=512, kernel_size=(4,4), strides=(2,2), padding='same', name='conv_t1')(conv7)
conv_t1_up = BilinearUpSampling2D(target_size=tuple(pool5.get_shape().as_list()[1:3]))(conv_t1)

stacked_1 = concatenate(inputs=[conv_t1_up, pool5], axis=-1, name='stacked_1')
fuse_1_1 = Conv2D(filters=512, kernel_size=(1,1), activation='relu', padding='same', name='fuse_1_1')(stacked_1)
fuse_1_2 = Conv2D(filters=512, kernel_size=(1,1), activation='relu', padding='same', name='fuse_1_2')(fuse_1_1)

conv_t2 = Conv2DTranspose(filters=256, kernel_size=(4,4), strides=(2,2), padding='same', name='conv_t2')(fuse_1_2)
conv_t2_up = BilinearUpSampling2D(target_size=tuple(pool4.get_shape().as_list()[1:3]))(conv_t2)

stacked_2 = concatenate(inputs=[conv_t2_up, pool4], axis=-1, name='stacked_2')
fuse_2_1 = Conv2D(filters=256, kernel_size=(1,1), activation='relu', padding='same', name='fuse_2_1')(stacked_2)
fuse_2_2 = Conv2D(filters=256, kernel_size=(1,1), activation='relu', padding='same', name='fuse_2_2')(fuse_2_1)

conv_t3 = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='conv_t3')(fuse_2_2)
conv_t3_up = BilinearUpSampling2D(target_size=tuple(pool3.get_shape().as_list()[1:3]))(conv_t3)

stacked_3 = concatenate(inputs=[conv_t3_up, pool3], axis=-1, name='stacked_3')
fuse_3_1 = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='same', name='fuse_3_1')(stacked_3)
fuse_3_2 = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='same', name='fuse_3_2')(fuse_3_1)

conv_t4 = Conv2DTranspose(filters=64, kernel_size=(4,4), strides=(2,2), padding='same', name='conv_t4')(fuse_3_2)
conv_t4_up = BilinearUpSampling2D(target_size=tuple(pool2.get_shape().as_list()[1:3]))(conv_t4)

stacked_4 = concatenate(inputs=[conv_t4_up, pool2], axis=-1, name='stacked_4')
fuse_4_1 = Conv2D(filters=64, kernel_size=(1,1), activation='relu', padding='same', name='fuse_4_1')(stacked_4)
fuse_4_2 = Conv2D(filters=64, kernel_size=(1,1), activation='relu', padding='same', name='fuse_4_2')(fuse_4_1)

# Final upscaling
output = Conv2DTranspose(filters=n_classes, kernel_size=(16,16), strides=(2,2), 
                         padding='same', name='output')(fuse_4_2)
output_up = BilinearUpSampling2D(target_size=tuple(input.get_shape().as_list()[1:3]))(output)

In [148]:
# ?BilinearUpSampling2D

In [149]:
tuple(pool5.get_shape().as_list()[1:3])

(16, 16)

In [150]:
test_model = Model(input=input, output=output_up)

/Users/teerapatjenrungrot/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bi...)`
  """Entry point for launching an IPython kernel.


In [151]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


# test_model.summary()
SVG(model_to_dot(test_model, show_shapes=True).create(prog='dot', format='svg'))
plot_model(test_model, show_shapes=True, to_file='model.png')

# Training 

## Load data

In [ ]:
class seg_dataset_reader:
    path = ""
    class_mappings = ""
    files = []
    images = []
    annotations = []
    test_images = []
    test_annotations = []
    batch_offset = 0
    epochs_completed = 0

    def __init__(self, deepscores_path, max_pages=40, crop=True, crop_size=[1000,1000], test_size=20):
        """
        Initialize a file reader for the DeepScores classification data
        :param records_list: path to the dataset
        sample record: {'image': f, 'annotation': annotation_file, 'filename': filename}
        """
        print("Initializing DeepScores Classification Batch Dataset Reader...")
        self.path = deepscores_path
        self.max_pages = max_pages
        self.crop = crop
        self.crop_size = crop_size
        self.test_size = test_size

        images_list = []
        images_glob = os.path.join(self.path, "images_png", '*.' + 'png')
        images_list.extend(glob.glob(images_glob))

        #shuffle image list
        shuffle(images_list)

        if max_pages is None:
            max_pages = len(images_list)
            import sys
            sys.exit(1)

        if max_pages > len(images_list):
            print("Not enough data, only " + str(len(images_list)) + " available")

        if test_size >= max_pages:
            print("Test set too big ("+str(test_size)+"), max_pages is: "+str(max_pages))
            import sys
            sys.exit(1)

        print("Splitting dataset, train: "+str(max_pages-test_size)+" images, test: "+str(test_size)+ " images")
        test_image_list = images_list[0:test_size]
        train_image_list = images_list[test_size:max_pages]

        # test_annotation_list = [image_file.replace("/images_png/", "/pix_annotations_png/") for image_file in test_image_list]
        # train_annotation_list = [image_file.replace("/images_png/", "/pix_annotations_png/") for image_file in train_image_list]

        self._read_images(test_image_list,train_image_list)

    def _read_images(self,test_image_list,train_image_list):

        dat_train = [self._transform(filename) for filename in train_image_list]
        for dat in dat_train:
            self.images.append(dat[0])
            self.annotations.append(dat[1])
        self.images = np.array(self.images)
        self.images = np.expand_dims(self.images, -1)

        self.annotations = np.array(self.annotations)
        self.annotations = np.expand_dims(self.annotations, -1)

        print("Training set done")
        dat_test = [self._transform(filename) for filename in test_image_list]
        for dat in dat_test:
            self.test_images.append(dat[0])
            self.test_annotations.append(dat[1])
        self.test_images = np.array(self.test_images)
        self.test_images = np.expand_dims(self.test_images, -1)

        self.test_annotations = np.array(self.test_annotations)
        self.test_annotations = np.expand_dims(self.test_annotations, -1)
        print("Test set done")


    def _transform(self, filename):
        image = misc.imread(filename)
        annotation = misc.imread(filename.replace("/images_png/", "/pix_annotations_png/"))
        print("im working!" + str(randint(0,10)))
        if not image.shape[0:2] == annotation.shape[0:2]:
            print("input and annotation have different sizes!")
            import sys
            import pdb
            pdb.set_trace()
            sys.exit(1)

        if image.shape[-1] != 1:
            # take mean over color channels, image BW anyways --> fix in dataset creation
            image = np.mean(image, -1)

        if self.crop:
            coord_0 = randint(0, (image.shape[0] - self.crop_size[0]))
            coord_1 = randint(0, (image.shape[1] - self.crop_size[1]))

            image = image[coord_0:(coord_0+self.crop_size[0]),coord_1:(coord_1+self.crop_size[1])]
            annotation = annotation[coord_0:(coord_0 + self.crop_size[0]), coord_1:(coord_1 + self.crop_size[1])]

        return [image, annotation]

    # from PIL import Image
    # im = Image.fromarray(image)
    # im.show()
    # im = Image.fromarray(annotation)
    # im.show()


    def get_records(self):
        return self.images, self.annotations

    def reset_batch_offset(self, offset=0):
        self.batch_offset = offset

    def get_test_records(self):
        return self.test_images, self.test_annotations

    def next_batch(self, batch_size):
        start = self.batch_offset
        self.batch_offset += batch_size
        if self.batch_offset > self.images.shape[0]:
            # Finished epoch
            self.epochs_completed += 1
            #             print("****************** Epochs completed: " + str(self.epochs_completed) + "******************")
            # Shuffle the data
            perm = np.arange(self.images.shape[0])
            np.random.shuffle(perm)
            self.images = self.images[perm]
            self.annotations = self.annotations[perm]
            # Start next epoch
            start = 0
            self.batch_offset = batch_size

        end = self.batch_offset
        return self.images[start:end], self.annotations[start:end]

    def get_random_batch(self, batch_size):
        indexes = np.random.randint(0, self.images.shape[0], size=[batch_size]).tolist()
        return self.images[indexes], self.annotations[indexes]

In [ ]:
data_reader = seg_dataset_reader('/home/mirlab/Downloads/DeepScores/',
                                 crop=True, crop_size=[250,250])

In [ ]:
NUM_ITERATION = 10000
PRINT_EVERY_ITR = 100
SAVE_EVERY_ITR = 1000

In [ ]:
hist = {
    'acc': [],
    'loss': [],
    'val_acc': [],
    'val_loss': [],
}

In [ ]:
for itr in range(NUM_ITERATION):
    train_img, train_annotation = data_reader.next_batch(20)
    val_img, val_annotation = data_reader.get_test_records()
    history = model.fit(x=train_img, y=train_annotation, verbose=False, validation_data=(val_img, val_annotation))
    for key in history.history:
        value = history.history[key]
        hist[key].append(value)
        
    if itr % PRINT_EVERY_ITR == 0:
        plt.close()
        print("Finish Iteration {:}".format(itr))
        plt.figure(figsize=(20,6))
        plt.subplot(1,2,1)
        plt.plot(hist['loss'], label='training')
        plt.plot(hist['val_loss'], label='validation')
        plt.title('Loss')
        plt.legend()
        plt.yscale('log')
        plt.xlabel('Epochs')

        plt.subplot(1,2,2)
        plt.plot(hist['acc'], label='training')
        plt.plot(hist['val_acc'], label='validation')
        plt.title('Accuracy')
        plt.legend()
        plt.xlabel('Epochs')

        plt.show()
    if itr % SAVE_EVERY_ITR == 0:
        outputFile = 'model-checkpoint-iteration-{:}.h5'.format(itr)
        print("Save model to {:}".format(outputFile))
        model.save(outputFile)

# Test model

## Load model

In [ ]:
from keras.models import load_model

model = load_model('model-checkpoint-iteration-36000.h5', custom_objects={
    'BilinearUpSampling2D': BilinearUpSampling2D,
    'CroppingLike2D': CroppingLike2D,
    'customLoss': customLoss,
})

In [ ]:
data_reader = seg_dataset_reader('./data', max_pages=1, test_size=0,
                                 crop=True, crop_size=[500,500])

## Train data

In [ ]:
train_img, train_annotation = data_reader.next_batch(20)

In [ ]:
train_img.shape

In [ ]:
prediction = np.argmax(model.predict(train_img), axis=-1) # get the argmax of 124 classes

In [ ]:
img_id = 0

In [ ]:
plt.figure(figsize=(12,20))
plt.subplot(1,3,1)
plt.imshow(train_img[img_id,:,:,0], cmap='gray')
plt.title("Input image")
plt.subplot(1,3,2)
plt.imshow(train_annotation[img_id,:,:,0], cmap='gray')
plt.title("Ground Truth annotation")
plt.subplot(1,3,3)
plt.imshow(prediction[img_id,:,:], cmap='gray')
plt.title("Semantic Prediction")

In [ ]:
ypred = np.argmax(model.predict(train_img), axis=-1) # get the argmax of 124 classes

In [ ]:
def myMetric(ytarget, ypred):
    num = 0
    denom = 0
    for img in range(ytarget.shape[0]):
        for row in range(ytarget.shape[1]):
            for col in range(ytarget.shape[2]):
                if ytarget[img,row,col,0] != 0 or ypred[img,row,col] != 0:
                    denom += 1
                    if ytarget[img,row,col,0] == ypred[img,row,col]:
                        num += 1
    return (num, denom)
    
myMetric(train_annotation, ypred)

In [ ]:
train_annotation.shape

## Test data

In [ ]:
test_img, test_annotation = data_reader.get_test_records()

In [ ]:
prediction = np.argmax(model.predict(test_img), axis=-1) # get the argmax of 124 classes

In [ ]:
img_id = 10

In [ ]:
plt.figure(figsize=(12,20))
plt.subplot(1,3,1)
plt.imshow(test_img[img_id,:,:,0], cmap='gray')
plt.title("Input image")
plt.subplot(1,3,2)
plt.imshow(test_annotation[img_id,:,:,0], cmap='gray')
plt.title("Ground Truth annotation")
plt.subplot(1,3,3)
plt.imshow(prediction[img_id,:,:], cmap='gray')
plt.title("Semantic Prediction")

In [ ]:
np.unique(prediction)

In [ ]:
np.unique(test_annotation[img_id,:,:,0])